In [ ]:
# Model Training:
# 1) Load all data from preprocessing (training/test splits, etc)
# 2) Begin Training Models
    #  a) Decision Tree
    #  b) Naive Bayes
    #  c) Logistic Regression
    #  d) SVM
# 3) Testing Models
# 4) New Iterations

In [1]:
# Imports
from Models import ModelUtil
from Data import Preprocessing, DataUtil
from Visualization import VisualUtil
from Logs import logging as logs
from sklearn.ensemble import AdaBoostClassifier

import importlib
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
import configparser
import numpy as np

config = configparser.ConfigParser()
config.read('Data//config.ini')

importlib.reload(Preprocessing)
importlib.reload(ModelUtil)
importlib.reload(VisualUtil)
importlib.reload(logs)

import warnings
warnings.filterwarnings("ignore")

In [3]:
# 1) Load all data from preprocessing 
importlib.reload(Preprocessing)

if("False" in config['DATA']['USE_NEW_PREPROCESSING']):
    infieldDataFrame, outfieldDataFrame = Preprocessing.dataFiltering([], False)

    # drop nan values from the used columns
    specific_columns = ["PitcherThrows", "BatterSide", "TaggedPitchType", "PlateLocHeight", "PlateLocSide", "ZoneSpeed", "SpinRate", "RelSpeed", "HorzBreak", "VertBreak"]
    specific_columns = ["PitcherThrows", "BatterSide", "TaggedPitchType", "RelSpeed", "VertRelAngle", "HorzRelAngle", "SpinRate", "SpinAxis", "RelHeight", "RelSide", "VertBreak", "InducedVertBreak", "HorzBreak", "VertApprAngle", "HorzApprAngle"] # pitcher averages
    
    infieldDataFrame = infieldDataFrame.dropna(axis=0, how='any',subset=specific_columns)

else:
    normDataFrame = Preprocessing.dataProcessing()
    infieldDataFrame, outfieldDataFrame = Preprocessing.dataFiltering(normDataFrame, True)

In [ ]:
infieldcorrmatrix = infieldDataFrame.corr()
infieldcorrmatrix.to_csv('Infield_Correlation_Matrix')
outfieldcorrmatrix = outfieldDataFrame.corr()
outfieldcorrmatrix.to_csv('Outfield_Correlation_Matrix')

In [9]:
importlib.reload(Preprocessing)

if("False" in config['DATA']['USE_NEW_PREPROCESSING']):
    Y = infieldDataFrame["FieldSlice"]
    X = infieldDataFrame[["PitcherThrows", "BatterSide", "TaggedPitchType", "PlateLocHeight", "PlateLocSide", "ZoneSpeed", "SpinRate", "RelSpeed", "HorzBreak", "VertBreak"]] 
    X = infieldDataFrame[["PitcherThrows", "BatterSide", "TaggedPitchType", "RelSpeed", "VertRelAngle", "HorzRelAngle", "SpinRate", "SpinAxis", "RelHeight", "RelSide", "VertBreak", "InducedVertBreak", "HorzBreak", "VertApprAngle", "HorzApprAngle"]] # pitcher averages
    X = DataUtil.normalizeData(X)
    xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size=0.25, random_state=11)
    # adb = AdaBoostClassifier()
    # adb_model = adb.fit(xTrain, yTrain)

    # calculate split information:
    trainingClassSplit = [0, 0, 0, 0, 0]
    for i in yTrain:
        trainingClassSplit[i-1] += 1

    
    testingClassSplit = [0, 0, 0, 0, 0]
    for i in yTest:
        testingClassSplit[i-1] += 1

    trainingClassPercent = []
    for i in trainingClassSplit:
        trainingClassPercent.append(round(i/len(yTrain), 4))

    testingClassPercent = []
    for i in testingClassSplit:
        testingClassPercent.append(round(i/len(yTest), 4))

    print("Training Class Splits (count, then percentage):")
    print(trainingClassSplit)
    print(trainingClassPercent)
    print("\nTesting Class Splits (count, then percentage):")
    print(testingClassSplit)
    print(testingClassPercent)
else:
    infieldY = infieldDataFrame[0][['Direction','Distance']]
    infieldX = infieldDataFrame[0][infieldDataFrame[1]] 
    if("True" in config['SPLIT']['TTS']):
        xTrain, xTest, yTrain, yTest = train_test_split(infieldX, infieldY, test_size=0.20, random_state=11)
        
    elif("True" in config['SPLIT']['KFold']):
        kf = KFold(n_splits=5, shuffle=True)
        for train_index, test_index in kf.split(infieldX):
            xTrain, xTest = infieldX.iloc[train_index,:], infieldX.iloc[test_index,:]
            yTrain, yTest = infieldY.iloc[train_index,:], infieldY.iloc[test_index,:]

    elif("True" in config['SPLIT']['LOOCV']):
        loo = LeaveOneOut()
        for train_index, test_index in loo.split(infieldX):
            xTrain, xTest = infieldX.iloc[train_index,:], infieldX.iloc[test_index,:]
            yTrain, yTest = infieldY.iloc[train_index,:], infieldY.iloc[test_index,:]

    else:
        print("No Splitting Method Selected")
        

# GroupKFold: (avoids putting data from the same group in the test set -- useful for Pitcher/Batter ID when we implement that.)

Training Class Splits (count, then percentage):
[7280, 10890, 10257, 8471, 4956]
[0.1739, 0.2602, 0.2451, 0.2024, 0.1184]

Testing Class Splits (count, then percentage):
[2444, 3583, 3391, 2813, 1721]
[0.1752, 0.2568, 0.243, 0.2016, 0.1234]


In [ ]:
# 2) Training Models

In [10]:
importlib.reload(ModelUtil)
importlib.reload(logs)
# a) Decision Tree
# Need to test these hyperparameters for best case
max_depth = 50
max_features = 30
max_leaf_nodes = 150
dtOutput = ModelUtil.runDT(xTrain, yTrain, xTest, yTest, max_depth, max_features, max_leaf_nodes)


training decision tree model...
done!
getting statistics...

printing statistics...
Model Type: DecisionTree

Training Size = 41854
Testing Size = 13952

Training Accuracy = 0.3566923113680891
Testing Accuracy = 0.3140768348623853

Training Average Error = 0.9359917809528361
Testing Average Error = 1.0002866972477065

Training Recall = [0.28365384615384615, 0.5868686868686869, 0.2188749146924052, 0.3847243536772518, 0.19552058111380144]
Testing Recall = [0.2397708674304419, 0.5266536421992744, 0.191094072544972, 0.34589406327763955, 0.1673445671121441]

Training f1 (micro, macro, weighted) = [0.35669231136808904, 0.33214547519096194, 0.37141881175102454]
Testing f1 (micro, macro, weighted) = [0.3140768348623853, 0.290809500307481, 0.3283532854897562]

Training auc (macro, weighted) = [0.9837080055304327, 0.9807428179295563]
Testing auc (macro, weighted) = [0.9832867295885249, 0.9798963953362626]

Hyper-Parameters: 

Max Tree Depth: 50
Max Tree Features: 30
Max Leaf Nodes: 150

Accuracy

In [7]:
importlib.reload(ModelUtil)
importlib.reload(logs)
# b) Naive Bayes

var_smoothing = 1e-9
nbOutput = ModelUtil.runNB(xTrain, yTrain, xTest, yTest, var_smoothing)

training Naive Bayes model...
done!
getting statistics...
printing statistics...
Model Type: NaiveBayes

Training Size = 41854
Testing Size = 13952

Training Accuracy = 0.2886701390548096
Testing Accuracy = 0.2907110091743119

Training Average Error = 1.0867300616428537
Testing Average Error = 1.086654243119266

Training Recall = [0.34395604395604396, 0.17061524334251607, 0.3463000877449547, 0.39015464526029986, 0.17413236481033093]
Testing Recall = [0.34083469721767595, 0.17945855428411944, 0.34915953995871424, 0.38464273018130113, 0.18245206275421266]

Training f1 (micro, macro, weighted) = [0.2886701390548096, 0.2792573277988354, 0.2964623169150509]
Testing f1 (micro, macro, weighted) = [0.2907110091743119, 0.28282215307660746, 0.297519100440032]

Training auc (macro, weighted) = [0.9852115934664859, 0.9803946941157787]
Testing auc (macro, weighted) = [0.9855514782258332, 0.9809961428373258]

Hyper-Parameters: 

Var Smoothing: 1e-09

Accuracy Score for Predicting on Training Data: 1

In [8]:
importlib.reload(ModelUtil)
importlib.reload(logs)
# c)Logistic Regression
lr = 0.8
e = 100
logRegOutput = ModelUtil.runLogReg(xTrain, yTrain, xTest, yTest, lr, e)

training logistic regression model...
done!
getting statistics...
printing statistics...
Model Type: LogisticRegression

Training Size = 41854
Testing Size = 13952

Training Accuracy = 0.33167678119176186
Testing Accuracy = 0.33034690366972475

Training Average Error = 0.9478663926984279
Testing Average Error = 0.9533400229357798

Training Recall = [0.18612637362637363, 0.6730027548209366, 0.07643560495271522, 0.48305985125723055, 0.06497175141242938]
Testing Recall = [0.1955810147299509, 0.6692715601451298, 0.08581539368917723, 0.47280483469605405, 0.06507844276583381]

Training f1 (micro, macro, weighted) = [0.33167678119176186, 0.2625481245002404, 0.3832079186044943]
Testing f1 (micro, macro, weighted) = [0.33034690366972475, 0.26547011124277786, 0.3789583609239713]

Training auc (macro, weighted) = [0.9719181355342176, 0.9597177948564394]
Testing auc (macro, weighted) = [0.9694441127666135, 0.9589056320126309]

Hyper-Parameters: 

Learning Rate: 0.8
Epochs: 100

Accuracy Score for 

In [10]:
importlib.reload(ModelUtil)
importlib.reload(logs)
# d) SVM
rC = 1
kernel='linear'
degree= 1
gamma= 'scale'
coef0= 0.0
svmOutput = ModelUtil.runSVM(xTrain, yTrain, xTest, yTest, rC, kernel, degree, gamma, coef0)

training SVM model...
done!
getting statistics...
logging statistics...


c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:71: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTestStats = dftest.groupby(["FieldSliceActual"]).size().reset_index()
c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:75: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTestStats["Correct"] = dftemp.groupby(["FieldSliceActual"]).size().reset_index()[0]
c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:77: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain cu

printing statistics...
Model Type: SVM

Training Size = 42213
Testing Size = 14072

Training Accuracy = 0.31333949257337784
Testing Accuracy = 0.32227117680500283

Training Average Error = 0.9562220169142207
Testing Average Error = 0.9404491188175099

Training Recall = [0.0, 0.8033190480532506, 0.0, 0.5206884356949193, 0.0]
Testing Recall = [0.0, 0.8188306340927807, 0.0, 0.5387018396390142, 0.0]

Training f1 (micro, macro, weighted) = [0.31333949257337784, 0.16828958750340162, 0.43098074084009164]
Testing f1 (micro, macro, weighted) = [0.32227117680500283, 0.17354902523824683, 0.44205483936829404]

Training auc (macro, weighted) = Error
Testing auc (macro, weighted) = Error

Hyper-Parameters: 

Regularization Constant: 1
Kernel Type: linear
Kernel Degree1
Kernel Coefficient (gamma): scale
Independent Term in Kernel (coef0): 0.0

Accuracy Score for Predicting on Training Data: 0.9562
Accuracy Score for Predicting on Test Data: 0.3223


c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:172: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTestStats = dftest.groupby(["FieldSliceActual"]).size().reset_index()
c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:176: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfTestStats["Correct"] = dftemp.groupby(["FieldSliceActual"]).size().reset_index()[0]
c:\Users\Trent\Desktop\Senior Design\shifting_model\Logs\logging.py:178: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain


Overall Average Probabilities
-------------------------------------
Section 1: 16.61%
Section 2: 26.53%
Section 3: 25.04%
Section 4: 20.27%
Section 5: 11.55%

Field Slice Counts for Training Data
--------------------------------------------------
Section	Truth	Prediction
1		7295		28007.0
2		10967		14206.0
3		10348		nan
4		8483		nan
5		5120		nan
Amount Correct: 13227
Amount Incorrect: 28986

Field Slice Counts for Testing Data
--------------------------------------------------
Section	Truth	Prediction
1		2513		9367.0
2		3643		4705.0
3		3427		nan
4		2881		nan
5		1608		nan
Amount Correct: 4535
Amount Incorrect: 9537
done!


In [ ]:
# z) RandomForestRegressor
for i in range(0, len(trainIn)):
    direction, distance = ModelUtil.runRFR(trainIn[i], trainOut[i], testIn[i], testOut[i])

In [72]:
# Change the value of index to look at different datapoints
importlib.reload(VisualUtil)
# 3) Model Testing:
dt = dtOutput[0]
nb = nbOutput[0]
logReg = logRegOutput[0]
# svm = svmOutput[0]

print("Testing Output: ")
# index of test value:
index = 6432
print(f"Actual Field Slice: \t\t{yTest.iloc[index]}")

print("\nDecision Tree:")
print(f"Predicted Field Slice: \t\t{dt.predict([xTest.iloc[index]])[0]}")
print(f"Field Slice Probabilities: \t{dt.predict_proba([xTest.iloc[index]])[0]}")

print("\nNaive Bayes:")
print(f"Predicted Field Slice: \t\t{nb.predict([xTest.iloc[index]])[0]}")
print(f"Field Slice Probabilities: \t{nb.predict_proba([xTest.iloc[index]])[0]}")

print("\nLogistic Regression:")
print(f"Predicted Field Slice: \t\t{logReg.predict([xTest.iloc[index]])[0]}")
print(f"Field Slice Probabilities: \t{logReg.predict_proba([xTest.iloc[index]])[0]}")

# print("\nSVM:")
# print(f"Predicted Field Slice: \t\t{svm.predict([xTest.iloc[index]])[0]}")
# print(f"Field Slice Probabilities: \t{svm.predict_proba([xTest.iloc[index]])[0]}")

averageProbs = dt.predict_proba([xTest.iloc[index]])[0] + nb.predict_proba([xTest.iloc[index]])[0] + logReg.predict_proba([xTest.iloc[index]])[0] # + svm.predict_proba([xTest.iloc[index]])[0]
averageProbs = averageProbs / 3 

print(f"\n\nAVG Prediction: \t\t{np.argmax(averageProbs)+1}")
print(f"Field Slice AVG Probabilities: \t{averageProbs}")

VisualUtil.visualizeData(averageProbs, [1])

Testing Output: 
Actual Field Slice: 		1

Decision Tree:
Predicted Field Slice: 		2
Field Slice Probabilities: 	[0.23381643 0.35942029 0.22995169 0.12173913 0.05507246]

Naive Bayes:
Predicted Field Slice: 		2
Field Slice Probabilities: 	[0.16966559 0.33103061 0.2822339  0.1707578  0.04631209]

Logistic Regression:
Predicted Field Slice: 		2
Field Slice Probabilities: 	[0.21869929 0.33894116 0.24981634 0.14661964 0.04592358]


AVG Prediction: 		2
Field Slice AVG Probabilities: 	[0.20739377 0.34313069 0.25400064 0.14637219 0.04910271]


In [73]:
# Gather data on average predictions
length = len(xTest)
correct = 0
incorrect = 0
wrongProbs = 0
wrongDistance = 0

dt = dtOutput[0]
nb = nbOutput[0]
logReg = logRegOutput[0]
# svm = svmOutput[0]
for index in range(length):
    averageProbs = dt.predict_proba([xTest.iloc[index]])[0] + nb.predict_proba([xTest.iloc[index]])[0] + logReg.predict_proba([xTest.iloc[index]])[0] #+ svm.predict_proba([xTest.iloc[index]])[0]
    averageProbs = averageProbs / 3

    actual = yTest.iloc[index]
    predicted = np.argmax(averageProbs)+1

    percentageActual = averageProbs[actual-1]

    if predicted == actual:
        correct += 1
    else:
        incorrect += 1 
        wrongProbs += percentageActual
        wrongDistance += abs(actual-predicted)


# Correct Prediction Count
print(correct)
# Incorrect Prediction Count
print(incorrect)
# Average probability of actual slice when guess is incorrect
print(wrongProbs/incorrect)
# Average distance from actual slice to guess slice when guess is incorrect
print(wrongDistance/incorrect)

4572
9380
0.207913457132854
1.4666311300639658


In [ ]:
# 4) Model Iterations and Improvements

In [307]:
# 5) Data Visualization

# Temporary method of getting percentages for testing purposes
infieldPercentages  = np.random.dirichlet(np.ones(4), size=1)[0]
outfieldPercentages = np.random.dirichlet(np.ones(1), size=1)[0]

VisualUtil.visualizeData(infieldPercentages, outfieldPercentages)
